In [0]:
!pip install quandl

In [0]:
import quandl
quandl.ApiConfig.api_key = 'K4cFx6oxnRuNQHdEy_j2'
import numpy as np
import pandas as pd
# Download financial data from Quandl/Sharadar
table = quandl.get_table('SHARADAR/SF1', paginate=True)
# Grab the most recent annual data ('MRY' denotes annual data)
stock_df = table

In [0]:
# Scrape S&P 500 tickers from Wikipedia
from bs4 import BeautifulSoup
import requests
import re
page_link = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
page_response = requests.get(page_link, timeout=1000)
page_content = BeautifulSoup(page_response.content, 'lxml')

In [0]:
# From web scraped Wikipedia content on the S&P 500, grab the ticker and industry for each firm
tickers = []
industries = []
for i, val in enumerate(page_content.find_all('tr')):
    if i > 0:
        try:
            # Here is where I grab the ticker
            tickers.append(val.find_all('a')[0].text)
            # Here is where I grab the industry
            industries.append(val.find_all('td')[3].text.replace('\n', ''))
        except:
            pass

In [0]:
# After ticker ZTS, the rest of the table entries are for acquisitions/mergers
last_pos = tickers.index('ZTS') + 1
sp_tickers = tickers[:last_pos]
sp_industries = industries[:last_pos]
# Create a new dataframe for S&P 500 and merge in Quandl data
sp_df = pd.DataFrame()
sp_df['tickers'] = sp_tickers
sp_df['industries'] = sp_industries
merged_df = sp_df.merge(stock_df, left_on='tickers', right_on='ticker', how='left')
merged_df= merged_df.loc[pd.isna(merged_df['tickers']) == 0 , : ]

In [0]:
merged_df.to_csv('sp500.csv', index=False)